<a href="https://colab.research.google.com/github/Ohm-T/PRO3600-CurseurOculaire/blob/main/PRO3600_Curseur_Oculaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources
Livrables & Compte-Rendu de réunion

Redirection vers les dossiers des documents: [accès en ligne](https://drive.google.com/drive/folders/1uq0HSjgD_PxCnNP869N8Rr-dH1hN9RYl?usp=sharing)

# Image Processing


In [ ]:
import cv2
import numpy as np


# Renvoie le vecteur déplacement de la pupille entre 2 images
def getPupilVector(lastPos, currentPos):
    return [lastPos[0] - currentPos[0], lastPos[1] - currentPos[1]]


# Renvoie la position de la pupille dans l'image
def getPupilPosition(image):
    # Extraction de l'imagette de l'oeil supérieur droit détecté
    eye_color = extractEyesPicture(image)
    # Condition de détection
    if eye_color is None:
        return [0, 0]

    # Passage en noir et blanc pour la réduction d'information et une meilleure détection
    gray = cv2.cvtColor(eye_color, cv2.COLOR_BGR2GRAY)

    # Détection de cercle dans l'imagette de l'oeil par la méthode de Hough
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)

    # Condition de détection d'un cercle
    if circles is None:
        return [0, 0]

    # Récupération de la taille de l'image
    rows, cols = gray.shape
    # Application d'un flou gaussien
    gray_blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    # Seuillage
    _, threshold = cv2.threshold(gray_blurred, 80, 255, cv2.THRESH_BINARY_INV)
    # Recherche des contours de l'oeil
    contours = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Conditions de détection de l'oeil
    if contours is None:
        return [0, 0]
    if len(contours) == 0:
        return [0, 0]

    # Récupération des coordonnées de la pupille
    (x, y, w, h) = cv2.boundingRect(contours[0])

    return [x + int(w / 2), y + int(h / 2)]


# Extrait une imagette de l'oeil supérieur droit
def extractEyesPicture(image):
    # Méthode des ondelettes de Haar pour la détection des yeux
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Définiton de l'image du visage en couleur
    roi_color, w, h = extractFacesPicture(image);
    # Condition de détection du visage
    if roi_color is None:
        return None
    # Passage en noir et blanc pour la réduction d'information et une meilleure détection
    gray = cv2.cvtColor(roi_color, cv2.COLOR_BGR2GRAY)
    # Détection des yeux
    eyes = eye_cascade.detectMultiScale(gray)

    # Définition de l'image extraite
    extracted = None
    # Bouclage sur les yeux détectés
    for (ex, ey, ew, eh) in eyes:
        # Condition de détection de l'oeil supérieur droit
        if ex < w / 2 - 50 and ey < h / 2 + 50:
            extracted = roi_color[ey:ey + eh, ex:ex + ew]

    return extracted


# Extrait une image englobant le visage de l'utilisateur
def extractFacesPicture(image):
    # Méthode des ondelettes de Haar pour la détection de visage
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Passage en noir et blanc pour la réduction d'information et une meilleure détection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Détection des visages
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # Conditions de détection du visage
    if faces is None:
        return None, None, None
    if len(faces) == 0:
        return None, None, None
    # Extraction des données sur l'image du premier visage détecté
    (x, y, w, h) = faces[0]
    # Extraction du 1er visage détecté sur l'image en couleur
    extracted = image[y:y + h, x:x + w]
    return extracted, w, h


# Renvoie une image cv2 de la caméra
def getCameraView(cameraSlot=0):
    cap = cv2.VideoCapture(cameraSlot)
    _, frame = cap.read()
    return frame


# Permet d'afficher à l'écran une image (utile pour le debug)
def showImage(image, imageName):
    cv2.imshow(imageName, image)

# Cursor Scaling


In [ ]:
import pyautogui
import ImageProcessing
import Main
import numpy as np
import Interface
import RoundButton
import tkinter as tk


# Applique la position au curseur vis-à-vis du vecteur
# déplacement de la pupille
def setCursorPosition(vector):
    pyautogui.moveTo(vector[0], vector[1])


# Constitue l'hypothèse de régression
# sur l'écran étalonné les valeurs étalons
# et optimisée
# theta défini le vecteur de paramétrisation
# x défini le vecteur d'entrer nouvelle
def hypothesis(theta, x):
    # On définie le vecteur sortie supposé
    y = 0

    # On construit y = sum(theta_i*x_i)
    for i in range(0, len(x)):
        y = theta[i] * x[i]
    return y


# Enregistre le point d'étalonnage en utilisation
# la dernière position de la pupille
def pointClickEvent(mouseClickEvent):
    # On récupère les coordonnées du curseur
    mouseX, mouseY = pyautogui.position()

    # On récupère les coordonnées de la pupille
    [eyeX, eyeY] = ImageProcessing.getPupilPosition(ImageProcessing.getCameraView())

    # On enregistre la réalisation d'étalonnage en prévision de la régression
    Main.registeredApproximations.append([mouseX, mouseY, eyeX, eyeY])

    # Condition définissant que tous les boutons ont été cliqués
    if len(Main.registeredApproximations) == len(Main.buttonsLocations):

        # Construction des matrice échantillon :
        # échantillons d'entrée : eyeX, eyeY
        X = []
        # échantillons de sortie : mouseX, mouseY
        YX = []
        YY = []
        for i in range(0, len(Main.registeredApproximations)):
            X.append([1, Main.registeredApproximations[i][2], Main.registeredApproximations[i][3]])
            YX.append[Main.registeredApproximations[i][0]]
            YY.append[Main.registeredApproximations[i][1]]

        # Transformation matricielle puis transposition pour coller à la formule de l'équation normale
        X = np.array(X)
        X.transpose()
        YX = np.array(YX)
        YX.transpose()
        YY = np.array(YY)
        YY.transpose()

        # Détermination des paramètres optimales pour les régressions
        thetaX = np.invert(X.transpose() * X) * X.transpose() * YX
        thetaY = np.invert(X.transpose() * X) * X.transpose() * YY

        # Enregistrement des vecteurs optimisés
        Main.approximation = [thetaX, thetaY]

        # Une fois les opérations effectuées, on affiche les boutons de contrôle :
        Interface.displayButtons()
        # On désactive le mode étalonnage
        Main.isScaling = False


# Liste les positions des boutons étalons et initialise buttonsLocations
def setPosition(xmax, ymax):
    side_width = 3
    diam = 100
    ray = diam // 2
    xmilieu = xmax // 2
    ymilieu = ymax // 2
    Main.buttonsLocations = [[0, 0], [0, ymilieu - ray], [0, ymax - diam], [xmilieu - ray, 0],
                             [xmilieu - ray, ymilieu - ray],
                             [xmilieu - ray, ymax - diam], [xmax - diam, 0], [xmax - diam, ymilieu - ray],
                             [xmax - diam, ymax - diam]]


# Affiche les points d'étalonnage à l'écran
def displayCalibratingPoints():
    # Appel d'une fenêtre tk
    fenetre = tk.Tk()
    # Mode fullscreen
    fenetre.attributes('-fullscreen', True)
    # Echap permet de quitter l'interface
    fenetre.bind('<Escape>', lambda e: fenetre.destroy())
    # Récupération des informations de l'écran
    width = fenetre.winfo_screenwidth()
    height = fenetre.winfo_screenheight()
    # création de la fenêtre
    canvas = tk.Canvas(fenetre, width=width, height=height, bg='black')
    canvas.pack()
    # initialisation des positions des boutons étalons
    setPosition(width, height)
    # Placement des boutons
    for i in range(len(Main.buttonsLocations)):
        button_i = RoundButton(fenetre, 100, 100, 50, 0, 'red', "black", command=Main.pointClickEvent())
        button_i.place(x=Main.buttonsLocations[i][0], y=Main.buttonsLocations[i][1])
        button_i.pack

    fenetre.mainloop()
    fenetre.destroy()


# Lance le mode étalonnage
def launchCalibrating():
    # On montre au programme qu'il est en mode étalonnage
    Main.isScaling = True
    # On reset les anciennes données de régression
    Main.registeredApproximations.clear()
    # On affiche les boutons étalons
    displayCalibratingPoints()

# Interface

In [ ]:
import tkinter as tk
from RoundButton import *


# Effectue les actions liées au bouton
def buttonClickEvent():
    return

# Affiche les boutons utilisateurs (exit & étalonnage)
def displayButtons():
    # Appel d'une fenêtre tk
    fenetre = tk.Tk()
    # Mode fullscreen
    fenetre.attributes('-fullscreen', True)
    # Echap permet de quitter l'interface
    fenetre.bind('<Escape>', lambda e: fenetre.destroy())
    # Récupération des informations de l'écran
    width = fenetre.winfo_screenwidth()
    height = fenetre.winfo_screenheight()
    # création de la fenêtre
    canvas = tk.Canvas(fenetre, width=width, height=height, bg='black')
    canvas.pack()
    # initialisation des positions des boutons étalons
    etan = tk.Button(fenetre, text='étannolage',command=displayCalibrationgPoints())
    etan.pack()
    quit = tk.Button(fenetre, text='quitter', command=fenetre.quit())
    quit.pack()
    fenetre.mainloop()
    fenetre.destroy()
    return None


# Round Button

In [ ]:
import tkinter as tk

class RoundButton(tk.Canvas):
    def __init__(self, parent, width, height, cornerradius, padding, color, bg, command=None):
        tk.Canvas.__init__(self, parent, borderwidth=0,
            relief="flat", highlightthickness=0, bg=bg)
        self.command = command

        if cornerradius > 0.5*width:
            print("Error: cornerradius is greater than width.")
            return None

        if cornerradius > 0.5*height:
            print("Error: cornerradius is greater than height.")
            return None

        rad = 2*cornerradius
        def shape():
            self.create_polygon((padding,height-cornerradius-padding,padding,cornerradius+padding,padding+cornerradius,padding,width-padding-cornerradius,padding,width-padding,cornerradius+padding,width-padding,height-cornerradius-padding,width-padding-cornerradius,height-padding,padding+cornerradius,height-padding), fill=color, outline=color)
            self.create_arc((padding,padding+rad,padding+rad,padding), start=90, extent=90, fill=color, outline=color)
            self.create_arc((width-padding-rad,padding,width-padding,padding+rad), start=0, extent=90, fill=color, outline=color)
            self.create_arc((width-padding,height-rad-padding,width-padding-rad,height-padding), start=270, extent=90, fill=color, outline=color)
            self.create_arc((padding,height-padding-rad,padding+rad,height-padding), start=180, extent=90, fill=color, outline=color)


        id = shape()
        (x0,y0,x1,y1)  = self.bbox("all")
        width = (x1-x0)
        height = (y1-y0)
        self.configure(width=width, height=height)
        self.bind("<ButtonPress-1>", self._on_press)
        self.bind("<ButtonRelease-1>", self._on_release)

    def _on_press(self, event):
        self.configure(relief="sunken")

    def _on_release(self, event):
        self.configure(relief="raised")
        if self.command is not None:
            self.command()

# Main

In [ ]:
import sched
import time
import CursorScaling
import ImageProcessing

# Donne le dernier vecteur position de l'oeil calculé
lastPupilPosition = [0, 0]

# Donne l'approximation calculée lors de l'échellonnage
approximation = []

# Liste des positions des points d'échelonnage sur l'écran
buttonsLocations = [[0, 0], [0, 30], [0, 60]]

# Liste des position de la pupille
# enregistrés lors de l'échelonnage avec les positions des points cibles
registeredApproximations = []

# Booléan indiquant si l'on est en mode étalonnage ou non
isScaling = False

# Lancement de l'étalonnage au démarrage du programme
CursorScaling.launchCalibrating()


# Définition de la fonction de gestion du curseur
def running():
    # Condition de mise à jour de la position du curseur
    if not isScaling:
        # On récupère la position de l'oeil
        [eyeX, eyeY] = ImageProcessing.getPupilPosition(ImageProcessing.getCameraView())
        # On en déduit la position prévue du curseur Les index 0,1 références thetaX et thetaY optimisant l'hypothèse
        # de régression et ont été calculés lors de l'étalonnage
        pos = [CursorScaling.hypothesis(approximation[0], eyeX), CursorScaling.hypothesis(approximation[1], eyeY)]
        # On déplace le curseur à cette position
        CursorScaling.setCursorPosition(pos)
        # On enregistre le dernier vecteur position de la pupille
        lastPupilPosition[0], lastPupilPosition[1] = pos[0], pos[1]


# Création de la boucle scheduler
s = sched.scheduler(time.monotonic, running)
# Lancement du scheduler
s.run()
